# Imports

In [ ]:
from pathlib import Path
import numpy as np
import torch
import math
from tqdm import tqdm
from torchvision.datasets import CocoDetection
import matplotlib.pyplot as plt
import torchvision

In [ ]:
from abbyy_course_cvdl_t3.coco_text import COCO_Text
from abbyy_course_cvdl_t3 import coco_evaluation
from abbyy_course_cvdl_t3.utils import evaluate_ap_from_cocotext_json
from abbyy_course_cvdl_t3.utils import dump_detections_to_cocotext_json

In [ ]:
cd ../task2

In [ ]:
import abbyy_course_cvdl_t2
from abbyy_course_cvdl_t2.convert import PointsToObjects, ObjectsToPoints
from abbyy_course_cvdl_t2.impl.train import train
from abbyy_course_cvdl_t2.impl.data import CocoTextDetection, CocoDetectionPrepareTransform
from abbyy_course_cvdl_t2.network import CenterNet
from abbyy_course_cvdl_t2.loss import CenterNetLoss

In [ ]:
cd ../task3

In [ ]:
device = torch.device('cuda:0')

# Data

In [ ]:
base = Path("/home/alexander/Downloads/coco2014/")
anno_path = base / 'cocotext.v2.json'
images_path = base / 'images/train2014'

assert anno_path.exists(), "Set your own path to annotation"
assert images_path.exists(), "Set your own path to images"


In [ ]:
ct = COCO_Text(anno_path)

In [ ]:
def draw_boxes(image_np, yc_t, xc_t, hy_t, wx_t, color=(0, 255, 0), thick=1):
    img = np.array(image_np)
    num_boxes = len(yc_t)
    val = np.array(color)
    for idx in range(num_boxes):
        yc = yc_t[idx]
        xc = xc_t[idx]
        hy = hy_t[idx]
        wx = wx_t[idx]
        img[
                yc - hy//2 - thick : yc - hy//2 + thick, 
                xc - wx//2 : xc + wx//2, 
        ] = val
        img[
                yc + hy//2 - thick : yc + hy//2 + thick, 
                xc - wx//2 : xc + wx//2, 
        ] = val

        img[
                yc - hy//2 : yc + hy//2, 
                (xc - wx//2 - thick): (xc - wx//2 + thick), 
        ] = val
        img[
                yc - hy//2 : yc + hy//2, 
                xc + wx//2 - thick: xc + wx//2 + thick, 
        ] = val
    return img
        
        

In [ ]:
n_classes=2
input_shape = (256, 256)

In [ ]:
ds_train = CocoTextDetection(
    Path("/home/alexander/Downloads/coco2014/images/train2014"),
    Path("/home/alexander/Downloads/coco2014/cocotext.v2.json"),
    transforms=CocoDetectionPrepareTransform(size=input_shape, #ids_map = {1: 0, 0: 0}
),
    area_fraction_threshold=1/32/32
)

In [ ]:
ds_val = CocoTextDetection(
    images_path,
    Path(anno_path),
    transforms=CocoDetectionPrepareTransform(size=input_shape, #ids_map = {1: 0, 0: 0}
    ),
    area_fraction_threshold=1/32/32,
    split='val'
)

In [ ]:
x, y = ds_train[6]

In [ ]:
y[:10]

In [ ]:
rnd = lambda x: torch.round(x).long()

plt.imshow(draw_boxes(
    x.permute(1, 2, 0).numpy(), rnd(y[:, 0]), rnd(y[:, 1]), rnd(y[:, 2]), rnd(y[:, 3]) 
))
#plt.imshow(x.permute(1, 2, 0))

In [ ]:
trainloader = torch.utils.data.DataLoader(
    ds_train, batch_size=2, shuffle=True
)

In [ ]:
valloader = torch.utils.data.DataLoader(
    ds_val, batch_size=1, shuffle=False
)

# Learning

In [ ]:
net = CenterNet(head_kwargs={'c_classes': n_classes}, nms_kwargs={'kernel_size': 5})
crit = CenterNetLoss(obj_to_points=ObjectsToPoints(num_classes=n_classes))

In [ ]:
net = train(ds_train, net=net, criterion=crit, batch_size=64, epochs=120, device=torch.device(device), lr=1e-3)

In [ ]:
torch.save(net, 'centernet.pth')

# Quality

## Preprocessing and visualization

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 8.0)

In [ ]:
net = torch.load('centernet.pth')
net.to(device)
net.eval()

In [ ]:
img_id = 101243
img_meta = ct.loadImgs(ids=[img_id])[0]
plt.imshow(plt.imread(images_path / img_meta['file_name']))
annIds = ct.getAnnIds(imgIds=img_meta['id'])
anns = ct.loadAnns(annIds)
ct.showAnns(anns)
plt.title(f"GT: {img_meta['id']}")


In [ ]:
anns

In [ ]:
img, anno = ds_val[ds_val.ids.index(str(img_id))]

In [ ]:
with torch.no_grad():
    preds = net(img[None, :].to(device))[0]

In [ ]:
def postprocess(data, scale=1.):
    unf = {}
    res = data[data[:, :, -1] > 0.2].detach()
    boxes = res[:, :4].clone()
    b = boxes.clone()
    boxes[:, 2] = (b[:, 2] - b[:, 0])
    boxes[:, 3] = (b[:, 3] - b[:, 1])
    boxes *= scale
    unf['boxes'] = boxes.numpy()
    unf['scores'] = res[:, -1].detach().numpy()
    return unf

In [ ]:
prepared_preds = []
img_ids = []

net.eval()

for num, img_id in enumerate(tqdm(ds_val.ids)):
    img_id = int(img_id)
    img_meta = ct.loadImgs(ids=[img_id])[0]
    with torch.no_grad():
        x = ds_val[num][0]
        pred = net(
            x[None, ...].to(device),
            return_objects=True
        )[0].detach().cpu().numpy()
        prepared_preds.append(
            postprocess(pred, scale=x.shape[1] / input_shape[1])
        )
        img_ids.append(img_id)


In [ ]:
prepared_preds[0]

Upscale maybe

In [ ]:
prpr = []
for i in prepared_preds:
    

## Saving to json

In [ ]:
scores = np.concatenate([u['scores'] for u in prepared_preds], axis=0)
boxes = np.concatenate([u['boxes'] for u in prepared_preds], axis=0)

In [ ]:
image_ids = []
for num, i in enumerate(img_ids):
    image_ids += [i] * len(prepared_preds[num]['boxes'])
image_ids = np.array(image_ids)

In [ ]:
print(len(image_ids.tolist()),
    len(boxes[:, 0].tolist()),
    len(boxes[:, 1].tolist()),
    len(boxes[:, 2].tolist()),
    len(boxes[:, 3].tolist()),
    len(scores.tolist()))

In [ ]:
dump_detections_to_cocotext_json(
    image_ids = image_ids.tolist(),
    xlefts=boxes[:, 0].tolist(),
    ytops=boxes[:, 1].tolist(),
    widths=boxes[:, 2].tolist(),
    heights=boxes[:, 3].tolist(),
    scores=scores.tolist(),
    path='predictions.json'
);
    

## AP calculation

In [ ]:
ap, prec, rec = evaluate_ap_from_cocotext_json(
    coco_text=ct,
    path='predictions.json'
)
print(f"Итоговый скор AP на val: {ap}")

In [ ]:
from matplotlib import pyplot as plt
plt.plot(prec, rec)
plt.xlabel('precision')
plt.ylabel('recall')
plt.title('PR curve')
plt.grid()